# ノードの再開

---

OpenHPC環境の停止したノードを再開します。

> このNotebookは AWS, AzureのVCノードに対してのみ利用可能です。他のクラウドプロバイダで実行中のVCノードはノード再開の機能をVCPがサポートしていないためエラーとなります。

## パラメータの指定

ノードを再開するのに必要となるパラメータを入力します。

### VCCアクセストークンの入力

VCノードを停止するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### グループ名

OpenHPCのUnitGroup名を指定します。

VCノードを作成時に指定したUnitGroup名を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

UnitGroupの変数をgroup_varsファイルから読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

## ノードの再開

現在のノードの状態を確認します。

In [ ]:
ug = vcp.get_ugroup(ugroup_name)
ug.df_nodes()

### マスターノードの再開

マスターノードを再開します。

> マスターノードを停止していない場合は、この節をスキップしてください。

In [ ]:
unit_master = ug.get_unit('master')
unit_master.power_on_nodes()

マスターノードの状態を確認します。`node_state` の表示が `RUNNING`となっていることを確認してください。

In [ ]:
unit_master.df_nodes()

NFSサーバが起動していることを確認します。

In [ ]:
!ansible {ugroup_name}_master -b -a 'systemctl status nfs-server'

上のセルがエラーとなり `Active: inactive (dead)` と表示されている場合はNFSサーバが起動していません。
そのような場合は次のセルのコメントを外して実行し、手動で起動できるかを確認してください。

In [ ]:
# !ansible {ugroup_name}_master -b -a 'systemctl restart nfs-server'

サービスを手動で起動した場合は、再度NFSサーバサービスの状態を確認します。次のセルのコメントを外して実行してください。

In [ ]:
# !ansible {ugroup_name}_master -b -a 'systemctl status nfs-server'

### 計算ノードの再開

計算ノードを再開します。

In [ ]:
unit_compute = ug.get_unit('compute')
unit_compute.power_on_nodes(num_power_on_nodes=gvars['compute_nodes'])

計算ノードの状態を確認します。全ノードの `node_state` の表示が `RUNNING`となっていることを確認してください。上のセルがエラーになった場合も次のセルを実行して各ノードの状態を確認してください。

In [ ]:
unit_compute.df_nodes()

一部のVCノードが `POWER_OFF`のままの場合は、次のセルのコメントを外して実行してください。その際に対象とするVCノードの`node_no`の値を指定してください。`node_no`の値は上のセルの出力結果で確認してください。

In [ ]:
# node_no =
# unit_compute.power_on_nodes(node_no=node_no)
# unit_compute.df_nodes()

## Slurmの状態を確認する

Slurmクラスタのノードの状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a sinfo

`STATE`が`idle`と表示されていればノードが利用可能な状態になっています。

もし、`STATE`が `down` となっているノードがある場合は次のセルのコメント(先頭の `#`)を外して実行してください。

> ノード停止中にインスタンスタイプを変更するなどの操作を行った場合は下のセルを実行しただけではSlurmクラスタを利用可能な状態にできないことがあります。
そのときは「031-設定ファイルの編集」または「032-設定ファイルの編集」のNotebookを用いて `slurm.conf` などの設定ファイルを修正してください。

In [ ]:
# for x in gvars['compute_etc_hosts'].values():
#     !ansible {ugroup_name}_master -b -a 'scontrol update nodename={x} state=resume' || true
#
# !ansible {ugroup_name}_master -a sinfo

`srun`コマンドで、`hostname` コマンドを計算ノードで実行させてみます。

計算ノードが２ノードの場合、以下のような実行結果が表示されます（ホストの表示順序は入れ替わることがあります）。

```
0: c1
1: c2
```

In [ ]:
!ansible {ugroup_name}_master -a 'srun -l -N {gvars["compute_nodes"]} hostname'